# 클래스 활성화의 히트맵 시각화하기

클래스 활성화 맵(CAM) 시각화 기법을 통해 컨브넷의 결정 과정을 디버깅할 수 있습니다. 이 기법은 입력 이미지에 대한 클래스 활성화의 히트맵을 만듭니다. 클래스 활성화 히트맵은 특정 출력 클래스에 대해 입력 이미지의 모든 위치에 대해 계산된 2D 점수 그리드입니다. 클래스에 대해 각 위치가 얼마나 중요한지를 알려 줍니다.

여기서 사용할 구체적인 구현은 [Grad-CAM: Visual Explanations from Deep Networks via Gradient-based Localization](https://arxiv.org/abs/1610.02391)에 기술되어 있는 것입니다. 입력 이미지가 주어지면 합성곱 층에 있는 특성 맵의 출력을 추출합니다. 그 다음 특성 맵의 모든 채널의 출력에 채널에 대한 클래스의 그래디언트 평균을 곱합니다. 

간단하게 소개하면,  **'입력 이미지가 클래스를 활성화하는 정도'에 대한 공간적인 맵을 만드는 것**입니다.

사전 훈련된 VGG16 네트워크를 다시 사용하여 이 기법을 시연해 보겠습니다.

초원을 걷는 어미와 새끼 아프리카 코끼리의 이미지를 적용해 보겠습니다:


![elephants](https://s3.amazonaws.com/book.keras.io/img/ch5/creative_commons_elephant.jpg)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications import imagenet_utils

from tensorflow.keras.models import Model
import tensorflow as tf
import numpy as np
import argparse
import imutils
import cv2

In [ ]:
PreTrained_Model = VGG16
# PreTrained_Model = ResNet50

In [ ]:
model = PreTrained_Model(weights="imagenet")

이 이미지를 VGG16 모델이 인식할 수 있도록 변환해보겠습니다. 이 모델은 224 × 224 크기의 이미지에서 훈련되었고 `imagenet_utils.preprocess_input` 함수에 있는 몇 가지 규칙에 따라 전처리 되었습니다. 

그러므로 이 이미지를 로드해서 224 × 224 크기로 변경하고 넘파이 float32 텐서로 바꾼 다음 이 전처리 함수를 적용해야 합니다.

In [ ]:
img_path = '../../datasets/creative_commons_elephant.jpg'

orig = cv2.imread(img_path)
resized = cv2.resize(orig, (224, 224))
image = cv2.cvtColor(resized, cv2.COLOR_BGR2RGB)

차원을 추가하여 (1, 224, 224, 3) 크기의 배치로 배열을 변환합니다.

In [ ]:
image = np.expand_dims(image, axis=0)

데이터를 전처리합니다(채널별 컬러 정규화를 수행합니다)

In [ ]:
image = imagenet_utils.preprocess_input(image)

추론을 수행하고, 추론 결과에서 가장 큰 인덱스를 찾습니다.

In [ ]:
preds = model.predict(image)
i = np.argmax(preds[0])

In [ ]:
decoded = imagenet_utils.decode_predictions(preds)
(imagenetID, label, prob) = decoded[0][0]
label = "{}: {:.2f}%".format(label, prob * 100)
print("[INFO] {}".format(label))

네트워크는 이 이미지가 아프리카 코끼리를 담고 있다고 인식했습니다. 

이미지에서 가장 아프리카 코끼리 같은 부위를 시각화하기 위해 Grad-CAM 처리 과정을 구현하겠습니다:

먼저, flatten을 수행하기 전의 레이어를 찾습니다.

In [ ]:
for layer in reversed(model.layers):
    # check to see if the layer has a 4D output
    if len(layer.output_shape) == 4:
        print(layer.name)
        layerName = layer.name
        break

(1) 사전 훈련 된 모델에 입력,    
(2) 네트워크에서 (아마도) 최종 4D 레이어의 출력,    
(3) 모델에서 softmax 활성화의 출력을 제공하여 그라디언트 모델을 구성    

In [ ]:
gradModel = Model(
    inputs=[model.inputs],
    outputs=[model.get_layer(layerName).output, model.output])

이미지 텐서를 float-32 데이터 유형으로 캐스팅하고, 그라디언트 모델을 통해 이미지를 전달하고, 특정 클래스 인덱스와 관련된 loss를 기록합니다.

In [ ]:
# record operations for automatic differentiation
with tf.GradientTape() as tape:
    inputs = tf.cast(image, tf.float32)
    (convOutputs, predictions) = gradModel(inputs)
    loss = predictions[:, i]

automatic differentiation 사용하여 그라디언트 계산

In [ ]:
grads = tape.gradient(loss, convOutputs)

In [ ]:
# compute the guided gradients
castConvOutputs = tf.cast(convOutputs > 0, "float32")
castGrads = tf.cast(grads > 0, "float32")
guidedGrads = castConvOutputs * castGrads * grads

convolution과 guided gradients는 배치 차원 (필요하지 않은)을 가지므로 배치 차원을 제거합니다.

In [ ]:
convOutputs = convOutputs[0]
guidedGrads = guidedGrads[0]

gradients 값의 평균을 계산하고, 이를 가중치로 사용하여 필터의 누적값을 계산합니다.

In [ ]:
weights = tf.reduce_mean(guidedGrads, axis=(0, 1))

In [ ]:
cam = tf.reduce_sum(tf.multiply(weights, convOutputs), axis=-1)

입력 이미지 크기와 일치하도록 출력 클래스 활성화 맵의 크기를 조정하십시오.

In [ ]:
(w, h) = (image.shape[2], image.shape[1])
heatmap = cv2.resize(cam.numpy(), (w, h))

모든 값이 [0, 1] 범위에 있도록 히트 맵을 정규화하고, 결과 값을 [0, 255] 범위로 스케일링한 다음 부호없는 8 비트 정수로 변환하십시오.

In [ ]:
numer = heatmap - np.min(heatmap)
denom = (heatmap.max() - heatmap.min()) + 1e-8
heatmap = numer / denom
heatmap = (heatmap * 255).astype("uint8")

In [ ]:
heatmap = cv2.resize(heatmap, (orig.shape[1], orig.shape[0]))

In [ ]:
alpha=0.7

heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
output = cv2.addWeighted(orig, alpha, heatmap, 1 - alpha, 0)

출력 이미지에 추론한 라벨과 confidence를 기록합니다.

In [ ]:
cv2.rectangle(output, (0, 0), (340, 40), (0, 0, 0), -1)
cv2.putText(output, label, (10, 25), cv2.FONT_HERSHEY_SIMPLEX,
            0.8, (255, 255, 255), 2);

원본이미지, heatmap, 둘을 합친 이미지를 쌓아서 한번에 모두 출력합니다.

In [ ]:
output = np.vstack([orig, heatmap, output])
output = imutils.resize(output, height=700)

In [ ]:
plt.figure(figsize=(10, 20))
plt.imshow(output[:,:,::-1])